In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import DirectoryLoader, PythonLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain

# Prepare .env and API Key before running the script below
load_dotenv()

True

In [15]:
# !pip install langchain-chroma

In [96]:
# load doc
# loader = DirectoryLoader(
#     # '../../../lightfm/tests', 
#     '../../../SVD_Compression/llm_test/test', 
#     glob="**/*.py", 
#     show_progress=True, 
#     #use_multithreading=True,
#     loader_cls=PythonLoader
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(docs)

# # vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
# # retriever = vectorstore.as_retriever()
# # context = retriever.invoke("How many test functions are there?")

# # define prompt and chat
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "Analyze the test functions from the codes below:\n\n{context}"),
#     MessagesPlaceholder(variable_name="messages")
# ])

# # chat = ChatOpenAI(model='gpt-4-turbo')
# chat = ChatOpenAI(model='gpt-4o', temperature=0.1)

# # combine prompt, chat and doc
# docs_chain = create_stuff_documents_chain(chat, prompt) 

# for chunk in docs_chain.stream({
#     "context": all_splits,
#     "messages": [
#         HumanMessage(content="How many test functions are there? Can you list them all?")
#     ],
# }):
#     print(chunk, end="", flush=True)

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2079.89it/s]


In [2]:
checklist_sample = '''
1. Does it contain test case to ensure models are loaded correctly without errors, checking for issues in model setup or initialization?
2. Does it contain test case to verify that the output dimensions and values from model predictions match expected outcomes?
3. Does it contain test case to confirm the accuracy and correctness of evaluation metrics used within the system, ensuring that metrics such as precision, recall, AUC, etc., are computed correctly?
4. Does it contain test case to evaluate the model’s performance over training to identify potential overfitting? This could involve comparing training and validation loss.
5. Does it contain test case to define and enforce performance thresholds for crucial metrics to guarantee model performance?
'''

In [3]:
# load doc
loader = DirectoryLoader(
    '../../../lightfm/tests', 
    # '../../../SVD_Compression/llm_test/test', 
    glob="**/*.py", 
    show_progress=True, 
    #use_multithreading=True,
    loader_cls=PythonLoader
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)

# define prompt and chat
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior machine learning engineer who specializes in performing Machine Learning system testing. Extract and analyze the test functions from the codes:\n\n{context}"),
    ("system", f"Here is the Machine Learning system testing checklist delimited by triple quotes '''{checklist_sample}'''"),
    MessagesPlaceholder(variable_name="messages")
])

# chat = ChatOpenAI(model='gpt-4-turbo')
chat = ChatOpenAI(model='gpt-4o', 
                  # temperature=0.1,
                 )

# combine prompt, chat and doc
docs_chain = create_stuff_documents_chain(chat, prompt) 

100%|████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 1604.17it/s]


In [4]:
for chunk in docs_chain.stream({
    "context": docs,
    "messages": [
        HumanMessage(content="""
        Evaluate whether the codes has fulfilled the requirements and deliver a completion score. Do not include a summary evaluation.
        Desired format:
        Checklist Evaluation:
            Requirement Title:
            Requirement:
            Observation:
            Evaluation:
        Completion Score:
        """)
    ],
}):
    print(chunk, end="", flush=True)

Checklist Evaluation:

1. Requirement Title: Model Loading and Initialization
   Requirement: Ensure models are loaded correctly without errors, checking for issues in model setup or initialization.
   Observation: The code includes tests for initializing the LightFM model and fitting it with various datasets.
   Evaluation: Fulfilled

2. Requirement Title: Output Dimensions and Values
   Requirement: Verify that the output dimensions and values from model predictions match expected outcomes.
   Observation: The code includes tests that validate predictions' values and shapes, ensuring they meet expected outcomes.
   Evaluation: Fulfilled

3. Requirement Title: Evaluation Metrics Accuracy
   Requirement: Confirm the accuracy and correctness of evaluation metrics used within the system, ensuring that metrics such as precision, recall, AUC, etc., are computed correctly.
   Observation: The code includes tests for precision_at_k, recall_at_k, and auc_score to validate that these metrics a